<a href="https://colab.research.google.com/github/Kaiziferr/Deep_Learning_Workshop/blob/master/CNN/01_workshop_convolucionales_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
X_train.shape

(60000, 28, 28)

In [4]:
#reshape to be sample with height channels
X_train = X_train.reshape((X_train.shape[0], 28, 28,1)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')

In [5]:
X_test.shape

(10000, 28, 28, 1)

In [6]:
#Normalize
X_train = X_train/255
X_test = X_test/255

In [7]:
#oncode
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [8]:
num_clases = y_test.shape[1]

Arquitectura

1. Primera capa oculta: capa convolucional Conv2D

  - Tiene 32 mapas de caracteristicas, con un tamaño de $5*5$ y una función de activación ReLu
Capa Pooling MaxPooling2D

2. Tamaño de pacht de $2 * 2$
  -Capa de regularización Dropout
3. Capa Flatten para conversión de la matriz2D en un vector
4. Capa Dense con 128 neuronas y la función de activación ReLu
5. Capa de salida con 10 neuronas para las 10 clases y una función de activación Softmax
6. La compilación con ADAM, perdidad loogaritmica y accuracy

In [11]:
def model_base():
  model = Sequential()
  model.add(Conv2D(32, (5,5), input_shape = (28,28,1), activation='relu'))
  model.add(MaxPooling2D())
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
  return model

In [12]:
y_train.shape

(60000, 10)

In [13]:
model = model_base()

In [14]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

Epoch 1/10
300/300 [==============================] - 46s 10ms/step - loss: 0.4918 - accuracy: 0.8627 - val_loss: 0.0783 - val_accuracy: 0.9769
Epoch 2/10
300/300 [==============================] - 2s 8ms/step - loss: 0.0766 - accuracy: 0.9764 - val_loss: 0.0497 - val_accuracy: 0.9845
Epoch 3/10
300/300 [==============================] - 2s 8ms/step - loss: 0.0546 - accuracy: 0.9833 - val_loss: 0.0399 - val_accuracy: 0.9869
Epoch 4/10
300/300 [==============================] - 2s 8ms/step - loss: 0.0394 - accuracy: 0.9876 - val_loss: 0.0362 - val_accuracy: 0.9875
Epoch 5/10
300/300 [==============================] - 2s 7ms/step - loss: 0.0309 - accuracy: 0.9903 - val_loss: 0.0357 - val_accuracy: 0.9884
Epoch 6/10
300/300 [==============================] - 2s 7ms/step - loss: 0.0290 - accuracy: 0.9905 - val_loss: 0.0369 - val_accuracy: 0.9875
Epoch 7/10
300/300 [==============================] - 2s 8ms/step - loss: 0.0207 - accuracy: 0.9934 - val_loss: 0.0316 - val_accuracy: 0.9898
Epoc

In [20]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Error de la CNN: %.2f%%" % (100-score[1]*100))

Error de la CNN: 1.16%


In [21]:
print('Accuracy :',score[1])
print('Loss :',score[0])

Accuracy : 0.9883999824523926
Loss : 0.03244349732995033


# CNN más profunda

In [22]:
def model_deep():
  model = Sequential()
  model.add(Conv2D(30, (5,5), input_shape=(28,28,1), activation='relu'))
  model.add(MaxPooling2D())
  model.add(Conv2D(15, (2,2), activation='relu'))
  model.add(MaxPooling2D())
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [23]:
model = model_deep()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

Epoch 1/10
300/300 [==============================] - 4s 10ms/step - loss: 0.8737 - accuracy: 0.7248 - val_loss: 0.0847 - val_accuracy: 0.9740
Epoch 2/10
300/300 [==============================] - 3s 9ms/step - loss: 0.1136 - accuracy: 0.9654 - val_loss: 0.0569 - val_accuracy: 0.9810
Epoch 3/10
300/300 [==============================] - 2s 8ms/step - loss: 0.0815 - accuracy: 0.9751 - val_loss: 0.0432 - val_accuracy: 0.9864
Epoch 4/10
300/300 [==============================] - 3s 9ms/step - loss: 0.0638 - accuracy: 0.9794 - val_loss: 0.0451 - val_accuracy: 0.9846
Epoch 5/10
300/300 [==============================] - 2s 8ms/step - loss: 0.0583 - accuracy: 0.9823 - val_loss: 0.0304 - val_accuracy: 0.9904
Epoch 6/10
300/300 [==============================] - 3s 8ms/step - loss: 0.0479 - accuracy: 0.9850 - val_loss: 0.0355 - val_accuracy: 0.9892
Epoch 7/10
300/300 [==============================] - 3s 8ms/step - loss: 0.0461 - accuracy: 0.9864 - val_loss: 0.0270 - val_accuracy: 0.9913
Epoch

In [24]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Error de la CNN profundo: %.2f%%" % (100-score[1]*100))

Error de la CNN profundo: 0.81%


In [25]:
print('Accuracy :',score[1])
print('Loss :',score[0])

Accuracy : 0.9919000267982483
Loss : 0.025170307606458664
